In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.__version__

'1.15.0'

In [3]:
def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

### Quintet Loss with Weights

In [4]:
## required for semi-hard triplet loss:
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes

def triplet_loss(vects):
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = vects[:, 1:]

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

def quintet_loss(vects):
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = vects[:, 1:]

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)
    
    mask_negatives = adjacency_not
    
    # pos 
    embed_pos = tf.matmul(mask_positives, embeddings)
    num_of_pos = tf.reduce_sum(mask_positives, axis=1, keepdims=True)
    centroid_embed_pos = tf.math.xdivy(embed_pos, num_of_pos)
    
    # add centroids to the batch
    embeddings_anchor_centroid_pos = tf.concat([embeddings, centroid_embed_pos], axis=0)
    
    # add label for centroids
    labels_pos = tf.concat([labels, labels], axis=0)
    labels_pos = tf.cast(labels_pos, dtype=dtypes.float32)
    vects_pos = tf.concat([tf.reshape(labels_pos, (-1, 1)), embeddings_anchor_centroid_pos], axis=1)
    return vects_pos

    # neg
    #return mask_negatives
    embed_neg = tf.matmul(mask_negatives, embeddings)
    num_of_neg = tf.reduce_sum(mask_negatives, axis=1, keepdims=True)
    centroid_embed_neg = tf.math.xdivy(embed_neg, num_of_neg)
    # add centroids to the batch
    embeddings_anchor_centroid_neg = tf.concat([embeddings, centroid_embed_neg], axis=0)
    # create the matrix of neg ids
    repeat = tf.fill([1, batch_size], 1)[0]
    neg_ids = tf.repeat(tf.reshape(labels, (1, -1)), repeats=[batch_size], axis=0)
    mask_negatives_bool = tf.cast(mask_negatives, dtype=dtypes.bool)
    neg_ids = tf.cast(neg_ids, dtype=dtypes.float32)
    # TODO: get the most frequent neg id
#     return tf.where(mask_negatives_bool, neg_ids, mask_negatives)
#     unique, _, count = tf.unique_with_counts(tf.where(mask_negatives_bool, neg_ids, mask_negatives))
#     max_occurrences = tf.reduce_max(count)
#     max_cond = tf.equal(count, max_occurrences)
    #max_numbers = tf.squeeze(tf.gather(unique, tf.where(max_cond)))
    #return max_cond
    neg_ids = tf.reduce_max(tf.where(mask_negatives_bool, neg_ids, mask_negatives), axis=1, keepdims=True)
    #return neg_ids
    labels_neg = tf.concat([tf.cast(labels, dtype=dtypes.float32), neg_ids], axis=0)
    #return labels_neg
    vects_neg = tf.concat([tf.reshape(labels_neg, (-1, 1)), embeddings_anchor_centroid_pos], axis=1)
#     return vects_neg
    
    #return triplet_loss(vects), triplet_loss(vects_pos), triplet_loss(vects_neg)
    #return triplet_loss(vects)
    return tf.reduce_mean([triplet_loss(vects), triplet_loss(vects_pos) * 0.5, triplet_loss(vects_neg) * 0.5])
    

v = tf.constant([[1.0, 1.0, 2.0], [2.0, 3.0, 4.0], [1.0, 1.0, 2.0], 
                 [1.0, 1.0, 2.0], [3.0, 3.0, 4.0], [3.0, 1.0, 2.0]]) # [2, 3.0, 4.0] [1, 1.0, 2.0]
tl_loss = quintet_loss(v)
sess=tf.Session() 
sess.run(tl_loss)

Instructions for updating:
Use `tf.cast` instead.


array([[1., 1., 2.],
       [2., 3., 4.],
       [1., 1., 2.],
       [1., 1., 2.],
       [3., 3., 4.],
       [3., 1., 2.],
       [1., 1., 2.],
       [2., 0., 0.],
       [1., 1., 2.],
       [1., 1., 2.],
       [3., 1., 2.],
       [3., 3., 4.]], dtype=float32)

(0.52771944, 1.0, 0.25, 0.018982518, 0.52272725, 0.8764739, 0.62, 0.10454356)

In [5]:
## required for semi-hard triplet loss:
import tensorflow as tf
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import numpy as np

def triplet_loss(vects):
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = vects[:, 1:]

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

def quintet_loss(inputs):
    margin = 1.
    labels = inputs[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = inputs[:, 1:]

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)
    
    mask_negatives = adjacency_not
    
    # Include the anchor to positives
#     mask_positives = math_ops.cast(adjacency, dtype=dtypes.float32)
    
#     return mask_positives
    
    # pos 
    embed_pos = tf.matmul(mask_positives, embeddings)
    num_of_pos = tf.reduce_sum(mask_positives, axis=1, keepdims=True)
    centroid_embed_pos = tf.math.xdivy(embed_pos, num_of_pos)
    
    # add centroids to the batch
    embeddings_anchor_centroid_pos = tf.concat([embeddings, centroid_embed_pos], axis=0)
    
    # add label for centroids
    labels_pos = tf.concat([labels, labels], axis=0)
    labels_pos = tf.cast(labels_pos, dtype=dtypes.float32)
    vects_pos = tf.concat([tf.reshape(labels_pos, (-1, 1)), embeddings_anchor_centroid_pos], axis=1)
#     return mask_positives

    # neg
    # create the matrix of neg ids
    repeat = tf.fill([1, batch_size], 1)[0]
    neg_ids = tf.repeat(tf.reshape(labels, (1, -1)), repeats=[batch_size], axis=0)
    mask_negatives_bool = tf.cast(mask_negatives, dtype=dtypes.bool)
    neg_ids = tf.cast(neg_ids, dtype=dtypes.float32)
    
    i = tf.constant(0)
    neg_matrix=tf.where(mask_negatives_bool, neg_ids, mask_negatives)

    neg_random_matrix = tf.Variable([])
    def random_negative(i, neg_random_matrix):
        batch = tf.gather(neg_matrix, i)
        batch = tf.boolean_mask(batch, tf.greater(batch, 0))
        prob = tf.random_uniform_initializer(minval=0., maxval=1.)(shape=[1])[0]
        value = tf.cast(prob * tf.cast(tf.size(batch), dtype=dtypes.float32), dtype=dtypes.int32)
        value = batch[value]
        neg_random_matrix = tf.concat([neg_random_matrix, [value]], axis=0)
        return [tf.add(i, 1), neg_random_matrix]
    _, neg_random_matrix = tf.while_loop(lambda i, _: i<batch_size, 
                                        random_negative, 
                                        [i, neg_random_matrix],
                                       shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None])])
    
    new_neg_ids = tf.reshape(neg_random_matrix, (batch_size, 1))
    mask_negatives = tf.cast(tf.equal(neg_ids, new_neg_ids), dtype=dtypes.float32)
#     return mask_negatives
    
    neg_ids = new_neg_ids
    
    # Embedding negs
    embed_neg = tf.matmul(mask_negatives, embeddings)
    num_of_neg = tf.reduce_sum(mask_negatives, axis=1, keepdims=True)
    centroid_embed_neg = tf.math.xdivy(embed_neg, num_of_neg)
    # add centroids to the batch
    embeddings_anchor_centroid_neg = tf.concat([embeddings, centroid_embed_neg], axis=0)
#     return neg_ids
#     return tf.where(mask_negatives_bool, neg_ids, mask_negatives)
    # Get the biggest id
#     neg_ids = tf.reduce_max(tf.where(mask_negatives_bool, neg_ids, mask_negatives), axis=1, keepdims=True)
#     return neg_ids
    labels_neg = neg_ids
    labels_neg = tf.concat([tf.cast(labels, dtype=dtypes.float32), neg_ids], axis=0)
#     return labels_neg
#     return embeddings_anchor_centroid_neg
    vects_neg = tf.concat([labels_neg, embeddings_anchor_centroid_neg], axis=1)
#     return vects_neg
    
    # Centroids embed
#     return triplet_loss(vects)
    embeddings = tf.concat([centroid_embed_pos, centroid_embed_neg], axis=0)
    labels = tf.cast(labels, dtype=dtypes.float32)
    labels = tf.concat([labels, neg_ids], axis=0)
    vects_centroids =  tf.concat([tf.reshape(labels, (-1, 1)), embeddings], axis=1)
    #return triplet_loss(vects), triplet_loss(vects_pos), triplet_loss(vects_neg)
    #return triplet_loss(vects)
    
#     return TL_w
    TL_anchor_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_centroid_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_pos_w = 1;0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_neg_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
#     TL_centroid_w = TL_w[0][3]
    #sum_of_w = tf.reduce_sum(TL_w)
    #TL_w = tf.truediv(TL_w, sum_of_w)
    # Normalization in probabilities
    #TL_anchor_w = tf.truediv(TL_anchor_w, sum_of_w)
    #TL_pos_w = tf.truediv(TL_pos_w, sum_of_w)
    #TL_neg_w = tf.truediv(TL_neg_w, sum_of_w)
#     return TL_pos_w, TL_neg_w
    TL = triplet_loss(inputs)
    TL_pos = triplet_loss(vects_pos)
    TL_neg = triplet_loss(vects_neg)
    TL_centroid = triplet_loss(vects_centroids)
#     TL_centroid = 0.0
    sum_of_median = tf.reduce_sum([TL * TL_anchor_w, TL_pos * TL_pos_w, TL_neg * TL_neg_w, TL_centroid * TL_centroid_w])  
    sum_of_weigths = TL_anchor_w + TL_pos_w + TL_neg_w + TL_centroid_w 
    weigthed_median = tf.truediv(sum_of_median, sum_of_weigths)    
    return tf.cast([weigthed_median, TL_anchor_w, TL_pos_w, TL_neg_w, TL_centroid_w, TL, TL_pos, TL_neg, TL_centroid], 
                   dtype=dtypes.float32)

def custom_loss(y_true, y_pred):
    return tf.reduce_mean(y_pred[0])

def TL_w_anchor(y_true, y_pred):
    return tf.reduce_mean(y_pred[1])
def TL_w_pos(y_true, y_pred):
    return tf.reduce_mean(y_pred[2])
def TL_w_neg(y_true, y_pred):
    return tf.reduce_mean(y_pred[3])
def TL_w_centroid(y_true, y_pred):
    return tf.reduce_mean(y_pred[4])
def TL(y_true, y_pred):
    return tf.reduce_mean(y_pred[5])
def TL_pos(y_true, y_pred):
    return tf.reduce_mean(y_pred[6])
def TL_neg(y_true, y_pred):
    return tf.reduce_mean(y_pred[7])
def TL_centroid(y_true, y_pred):
    return tf.reduce_mean(y_pred[8])

v = tf.constant([[1.0, 1.0, 2.0], [2.0, 3.0, 4.0], [1.0, 1.0, 2.0], 
                 [1.0, 1.0, 2.0], [3.0, 3.0, 4.0], [3.0, 1.0, 2.0]]) # [2, 3.0, 4.0] [1, 1.0, 2.0]
tl_loss = quintet_loss(v)
sess=tf.Session() 
sess.run(tl_loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


array([0.48923445, 1.        , 1.        , 1.        , 1.        ,
       0.25      , 0.52272725, 0.15789473, 1.0263158 ], dtype=float32)

In [13]:
l = [130926,133860,284787,102727,275267,352297,159354,295390,381207,321088
,351176,411703,250243,46067,190119,125376,424120,298717,291029,152442
,301724,36328,183961,416809,3407,144150,329419,374624,269326,410734
,374624,65934,220763,14679,13556,301593,184909,295390,76922,40227
,169079,31064,2859,200295,374624,153304,374624,159354,332879,257318
,424120,30610,350370,236970,168880,3407,47818,128387,188385,111626
,374624,58431,13556,344913,194533,198282,117300,104540,301724,198282
,240969,240969,187251,102727,231616,2644,95462,240969,168880,3407
,295390,86523,286212,301724,353652,410919,89993,295390,102709,59503
,130926,89899,144029,215227,73860,374624,308040,351509,139379,159354
,364270,220763,145285,275267,329419,167087,371122,86523,341677,130926
,139379,374624,45216,198282,243689,71124,30567,374624,194533,155740
,109864,36328,301724,63803,112696,70655,223497,189093,139379,376285
,109864,267661,374624,194533,325238,194533,125851,125851,345992,275267
,313536,134427,260398,130926,295390,295390,128522,329419,128387,3407
,47818,301724,388838,11160,304048,176043,413602,41105,174272,380477
,190119,66629,240969,315735,16190,34991,396897,235970,295390,301724
,159354,45013,60696,301724,49492,156529,280748,41105,295390,63803
,104110,72820,349524,395469,302654,240969,87472,301724,295390,422419
,236970,413602,81152,3407,294185,198282,269326,125376,235970,319605
,152246,349524,281898,156529,24529,374624,280748,410734,301724,237570
,122711,342186,394820,142540,80754,111480,93274,269326,368894,60696
,236970,194533,342186,31064,45376,329419,89993,376285,166836,269326
,348037,236970,301724,54423,3407,329419,116849,301724,220698,364510
,255216,371122,145685,60976,301724,23617,304048,298098,337080,275267
,48054,167758,172761,80754,424120,301927,374624,116849,245911,54423
,73860,295390,3407,71711,2859,234829,169079,86523,424120,295390
,81152,152442,134452,130926,383237,70655,315735,396897,367093,34991
,99231,104540,351487,167007,284961,240969,244632,63739,152442,210004
,356883,66629,37874,329419,380477,301724,172761,325238,291029,202337
,321088,101168,260398,375861,199307,104110,376609,99231,325238,46067
,128442,3407,81134,177017,80754,339137,45216,202337,128398,194533
,340685,243689,144029,152442,112955,261495,333917,301724,67049,410928
,175658,267661,198282,358948,333917,374624,3407,45376,76922,135763
,130926,114197,58431,200295,348037,295390,223497,301724,325238,165031
,194533,139379,206446,398356,301724,80754,198282,87472,329419,288151
,261305,134427,194533,300212,49492,169032,86523,366897,71124,292091
,160241,144150,126587,417858,167087,160241,93274,301294,187779,301724
,301724,329419,136890,18084]

import numpy as np

l = np.concatenate([np.asarray(np.reshape(l, (-1, 1)), np.int32), np.random.rand(len(l), 901) ], 1)

l

array([[1.30926000e+05, 1.60398836e-02, 4.98874287e-01, ...,
        3.14803366e-01, 5.87878146e-01, 9.02188455e-01],
       [1.33860000e+05, 3.07970113e-01, 3.66614283e-01, ...,
        4.84030796e-01, 5.01390221e-02, 1.66062161e-01],
       [2.84787000e+05, 9.21228908e-01, 7.22928016e-02, ...,
        8.65632646e-01, 3.49485702e-01, 1.81379350e-01],
       ...,
       [3.29419000e+05, 7.52911527e-01, 9.59667405e-01, ...,
        7.65883767e-01, 7.75899734e-01, 2.04349029e-01],
       [1.36890000e+05, 7.63322091e-01, 2.48772197e-01, ...,
        9.26766755e-01, 8.16594491e-01, 1.44814338e-02],
       [1.80840000e+04, 7.26801138e-01, 1.77437418e-01, ...,
        3.83668102e-01, 1.70476073e-01, 8.59313968e-01]])

In [22]:
import os
import _pickle as pickle

DIR = 'data/processed/{}/{}'.format('openoffice', 'bert')

with open(os.path.join(DIR, 'train_batch'), 'rb') as f:
    l = pickle.load(f)
    
l = l[3]

In [23]:
len(l)

384

In [24]:
np.asarray(np.reshape(l, (-1, 1)), np.int32)

array([[ 91607],
       [107375],
       [ 39602],
       [ 53755],
       [ 56186],
       [ 67087],
       [ 56186],
       [ 83937],
       [ 93798],
       [ 56186],
       [ 40863],
       [106079],
       [ 56186],
       [ 82164],
       [ 73887],
       [ 40738],
       [ 96812],
       [ 56186],
       [ 98409],
       [ 15050],
       [ 40046],
       [ 51115],
       [ 86046],
       [  9597],
       [ 42886],
       [ 90862],
       [ 56186],
       [ 76604],
       [  9843],
       [ 56186],
       [ 83937],
       [ 76604],
       [ 56186],
       [ 85165],
       [ 72865],
       [ 92807],
       [ 79871],
       [ 76604],
       [ 94406],
       [ 56186],
       [ 90509],
       [ 41884],
       [ 70261],
       [ 72865],
       [ 56186],
       [ 37264],
       [ 66937],
       [ 96812],
       [ 69006],
       [ 56186],
       [ 97157],
       [ 56186],
       [ 45972],
       [ 93322],
       [ 56186],
       [ 56186],
       [ 56186],
       [ 45743],
       [101956

In [25]:
train_batch = np.concatenate([np.asarray(np.reshape(l, (-1, 1)), np.int32), np.random.rand(len(l), 900) ], 1)

In [27]:
train_batch.shape

(384, 901)

In [28]:
## required for semi-hard triplet loss:
import tensorflow as tf
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import numpy as np

def triplet_loss(vects):
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = tf.cast(vects[:, 1:], dtype='float32')

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

def quintet_loss(inputs):

    margin = 1.
    labels = inputs[:, :1]

    labels = tf.cast(labels, dtype='int32')

    embeddings =  tf.cast(inputs[:, 1:], dtype='float32')

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    mask_negatives = adjacency_not

    # Include the anchor to positives
    mask_positives_centroids = math_ops.cast(adjacency, dtype=dtypes.float32)

#     return mask_positives

    # pos 
    embed_pos = tf.matmul(mask_positives_centroids, embeddings)
    num_of_pos = tf.reduce_sum(mask_positives_centroids, axis=1, keepdims=True)
    centroid_embed_pos = tf.math.xdivy(embed_pos, num_of_pos)
    labels_pos = tf.cast(labels, dtype=dtypes.float32)
    # negs
    embed_neg = tf.matmul(mask_negatives, embeddings)
    num_of_neg = tf.reduce_sum(mask_negatives, axis=1, keepdims=True)
    centroid_embed_neg = tf.math.xdivy(embed_neg, num_of_neg)

#     return mask_positives_centroids
    i = tf.constant(0)
    batch_centroid_matrix = tf.Variable([])
    batch_centroid_matrix_neg = tf.Variable([])
    batch_centroid_matrix_all = tf.Variable([])
    def iter_centroids(i, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all):
        # centroid pos
        mask_positives_batch = tf.reshape(tf.gather(mask_positives, i), (-1, 1))
        centroid_pos = tf.gather(centroid_embed_pos, i)
        
        centroid_embed = tf.repeat([centroid_pos], repeats=[batch_size], axis=0)
        new_batch_centroid_pos = mask_positives_batch * centroid_embed
        new_batch_embeddings = tf.cast(tf.logical_not(tf.cast(mask_positives_batch, 'bool')), 'float32') * embeddings 
        new_batch = tf.reduce_sum([new_batch_centroid_pos, new_batch_embeddings], axis=0, keepdims=True)[0]
        
        vects_new_batch = tf.concat([labels_pos, new_batch], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch)
        batch_centroid_matrix = tf.concat([batch_centroid_matrix, [TL_new_batch]], axis=0) 
        
        # centroid neg
        centroid_neg = tf.gather(centroid_embed_neg, i)
        mask_negatives_batch = tf.reshape(tf.gather(mask_negatives, i), (-1, 1))
        
        centroid_embed = tf.repeat([centroid_neg], repeats=[batch_size], axis=0)
        new_batch_centroid_neg = mask_negatives_batch * centroid_embed
        new_batch_embeddings = tf.cast(tf.logical_not(tf.cast(mask_negatives_batch, 'bool')), 'float32') * embeddings 
        new_batch = tf.reduce_sum([new_batch_centroid_neg, new_batch_embeddings], axis=0, keepdims=True)[0]
        
        vects_new_batch = tf.concat([labels_pos, new_batch], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch)
        batch_centroid_matrix_neg = tf.concat([batch_centroid_matrix_neg, [TL_new_batch]], axis=0) 
        
        # centroid pos and neg
        new_batch_centroids = tf.reduce_sum([new_batch_centroid_pos, new_batch_centroid_neg], axis=0, keepdims=True)[0]
        vects_new_batch_centroids = tf.concat([labels_pos, new_batch_centroids], axis=1)
        TL_new_batch = triplet_loss(vects_new_batch_centroids)
        batch_centroid_matrix_all = tf.concat([batch_centroid_matrix_all, [TL_new_batch]], axis=0)
        
        print(batch_centroid_matrix)
        
        return [tf.add(i, 1), batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all]
    _, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all = tf.while_loop(lambda i, a, b, c: i<batch_size, 
                                        iter_centroids, 
                                        [i, batch_centroid_matrix, batch_centroid_matrix_neg, batch_centroid_matrix_all],
                                       shape_invariants=[i.get_shape(),
                                                   tf.TensorShape([None]), tf.TensorShape([None]), tf.TensorShape([None])])

    TL_anchor_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_pos_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_neg_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    TL_centroid_w = 1.0 # tf.random_uniform_initializer(minval=0.0, maxval=1.)(shape=[1])[0]
    
#     tl_weights = tf.truediv(num_of_pos, tf.reduce_max(num_of_pos))
#     tl_w = tl_weights # tf.random_uniform_initializer(minval=0.0, maxval=tl_weights)(shape=[1])
#     TL_pos_weighted = tf.reshape(batch_centroid_matrix, (-1, 1)) * tl_w
#     TL_pos = tf.truediv(tf.reduce_sum(TL_pos_weighted), tf.reduce_sum(tl_w))

    TL = triplet_loss(inputs)
    TL_pos = tf.reduce_mean(batch_centroid_matrix)
    TL_neg = tf.reduce_mean(batch_centroid _matrix_neg) #triplet_loss(vects_neg)
    TL_centroid = tf.reduce_mean(batch_centroid_matrix_all) # triplet_loss(vects_centroids)

    sum_of_median = tf.reduce_sum([TL * TL_anchor_w, TL_pos * TL_pos_w, TL_neg * TL_neg_w, TL_centroid * TL_centroid_w]) # 
    sum_of_weigths = TL_anchor_w + TL_pos_w + TL_neg_w + TL_centroid_w
    weigthed_median = tf.truediv(sum_of_median, sum_of_weigths)    
    return tf.cast([weigthed_median, TL_anchor_w, TL_pos_w, TL_neg_w, 
                    TL_centroid_w, TL, TL_pos, TL_neg, TL_centroid], 
                   dtype=dtypes.float32)

v = tf.constant(train_batch) # [2, 3.0, 4.0] [1, 1.0, 2.0]
tl_loss = quintet_loss(v)
sess=tf.Session() 
sess.run(tl_loss)

Tensor("while_3/concat_1:0", shape=(?,), dtype=float32)


array([ 4.667259  ,  1.        ,  1.        ,  1.        ,  1.        ,
        0.9116882 ,  0.7158266 , 16.451548  ,  0.58997315], dtype=float32)